In [2]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import csv
import json
import re

In [14]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»', '&nbsp': ' ', u'\xa0': ' ',
                '\n': ' ', '\t': ' ', '    ': '', '&emsp': ' ',
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        # str = str.replace(k[1:], v)
    str = str.strip('\n')
    str = str.strip(' ')

    return str

def to_CSV(totaldata):
    pd.DataFrame(totaldata).to_csv('./STTRResult/01_STTR_phase2.csv', index=False, encoding="utf-8")
    
def ReadFiles(Directory_path):  # Read all the html files
    path = Directory_path  # The directory
    file_names = os.listdir(path)  # 得到文件夹下的所有文件名称
    files_position = []
    for file_name in file_names:
        position = path + "/" + file_name
        # print(position)
        files_position.append(position)

    print("Total number of HTML files: ", len(files_position))
    return files_position


def MultipleFileProcess(files_position):
    totaldata = []
    for position in files_position:
        data = SingleHTMLProcess(position)
        totaldata.append(data)

    return totaldata

In [15]:
def SingleHTMLProcess(path):
    dic = {"Proposal Number": "", "Phase 1 Contract #": "", "Subtopic Title": "", "Proposal Title": "",
           "Small Business Concern_Firm": "", "Small Business Concern_Address": "", "Small Business Concern_Phone": "",
           "Principal Investigator_Name": "", "Principal Investigator_E-mail": "", "Principal Investigator_Address": "",
           "Principal Investigator_Phone": "", "Principal Investigator_Company": "",
           "Business Official_Name": "", "Business Official_E-mail": "", "Business Official_Address": "",
           "Business Official_Phone": "",
           "TRL_Begin": "", "TRL_End": "", "Technical Abstract": "", "Potential NASA applications": "",
           "Potential non-NASA applications": "","Technology Taxonomy Mapping":"", 'Research Topic':"", 'Research Institution Name':"", 'Research Institution Address':"",
          "Research Institution Phone":""}
    htmlfile = open(path, 'r', encoding='utf-8')
    html=htmlfile  
    bs = BeautifulSoup(html, "lxml")  # 缩进格式
    print(path)
    dic['Technical Abstract'] = filterHTMLstr(getTechAbstract(bs))
    dic['Potential NASA applications'] = filterHTMLstr(getNonNASAApplication(bs))
    dic['Proposal Number'] = filterHTMLstr(getProposalNumber(bs))
    dic['Research Topic'] = filterHTMLstr(getResearchTopic(bs))
    dic['Proposal Title'] = filterHTMLstr(getProjectTitle(bs))
    dic['Phase 1 Contract #'] = filterHTMLstr(getPhase1Num(bs))
    smbAndRIInfo = getSMBAndRIInfo(bs)
    dic['Small Business Concern_Firm'] = filterHTMLstr(smbAndRIInfo[0])
    dic['Small Business Concern_Address'] = filterHTMLstr(smbAndRIInfo[1])
    dic['Small Business Concern_Phone'] = filterHTMLstr(smbAndRIInfo[2])
    dic['Research Institution Name'] = filterHTMLstr(smbAndRIInfo[3])
    dic['Research Institution Address'] = filterHTMLstr(smbAndRIInfo[4])
    dic['Research Institution Phone'] = filterHTMLstr(smbAndRIInfo[5])
    dic['Principal Investigator_Name'] = filterHTMLstr(getInvestigatorInfo(bs)[0])
    dic['Principal Investigator_Company'] = filterHTMLstr(getInvestigatorInfo(bs)[1])
    return dic

In [31]:
Directory_path = "../Datasets/01/sttr/phase2"
files_position = ReadFiles(Directory_path)
totaldata = MultipleFileProcess(files_position)
to_CSV(totaldata)

Total number of HTML files:  9
../Datasets/01/sttr/phase2/STTR-01-2-02-010041.html
../Datasets/01/sttr/phase2/STTR-01-2-03-010052.html
../Datasets/01/sttr/phase2/STTR-01-2-02-010066.html
../Datasets/01/sttr/phase2/STTR-01-2-02-010049.html
../Datasets/01/sttr/phase2/STTR-01-2-03-010037.html
../Datasets/01/sttr/phase2/STTR-01-2-04-010032.html
../Datasets/01/sttr/phase2/STTR-01-2-01-010009.html
../Datasets/01/sttr/phase2/STTR-01-2-04-010053.html
../Datasets/01/sttr/phase2/STTR-01-2-01-010065.html


# Test

In [25]:
path = "../Datasets/01/sttr/phase2/STTR-01-2-02-010041.html"
htmlfile = open(path, 'r', encoding='utf-8')
html=htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式

In [8]:
### Technical Abstract and Potential Commercial Application
def getTechAbstract(bs):
    technicalAbstract = bs.find_all("p")[2].get_text()
    return technicalAbstract

def getNonNASAApplication(bs):
    nonNASAApplication = bs.find_all("p")[3].get_text()
    return nonNASAApplication

In [9]:
### Proposal Number
def getProposalNumber(bs):
    proposalNumber = bs.find_all("tr")[0].find_all("td")[1].get_text().strip()
    return proposalNumber

### Phase 1 contract#
def getPhase1Num(bs):
    phase1Number = bs.find_all("tr")[1].find_all("td")[1].get_text().strip()
    return phase1Number

In [10]:
### Research Topic and project title
def getResearchTopic(bs):
    researchTopic = bs.find_all("tr")[2].find_all("td")[1].get_text().strip()
    return researchTopic

def getProjectTitle(bs):
    projectTitle = bs.find_all("tr")[3].find_all("td")[1].get_text().strip()
    return projectTitle

In [11]:
### Small Business and Investigator Info
def getSMBAndRIInfo(bs):
    table = bs.find_all("table")[1]
    regNames = re.compile(r'NAME: *(.*?) *\n', re.S)
    names = re.findall(regNames, table.get_text())
    smbName = names[0]
    riName = names[1]

    regAddresses = re.compile(r'ADDRESS: *(.*?) *\n', re.S)
    addresses = re.findall(regAddresses, table.get_text())
    smbAddress = addresses[0]
    riAddress = addresses[1]

    regCities = re.compile(r'CITY: *(.*?) *\n', re.S)
    cities = re.findall(regCities, table.get_text())
    smbCity = cities[0]
    riCity = cities[1]

    regStates = re.compile(r'STATE/ZIP: *(.*?) \xa0', re.S)
    states = re.findall(regStates, table.get_text())
    smbState = states[0]
    riState = states[1]

    regZips = re.compile(r'STATE/ZIP:.*? \xa0 *(.*?)\n', re.S)
    zips = re.findall(regZips, table.get_text())
    smbZip = zips[0]
    riZip = zips[1]

    regPhones = re.compile(r'PHONE: *(.*?) *\n')
    phones = re.findall(regPhones, table.get_text())
    smbPhone = phones[0]
    riPhone = phones[1]
    
    smbAddress = smbAddress + ", " + smbCity + ", " + smbState + ", " + smbZip
    riAddress = riAddress + ", " + riCity + ", " + riState + ", " + riZip
    return [smbName, smbAddress, smbPhone, riName, riAddress, riPhone]

In [29]:
def getInvestigatorInfo(bs):
    table = bs.find_all("table")[2].find_all("th")[0].get_text()
    regPIInfo = re.compile(r'\xa0(.*?) *\n')
    piInfo = re.findall(regPIInfo, table)
    piName = piInfo[0]
    piCompany = piInfo[1] if len(piInfo) > 1 else  ""
    return [piName, piCompany]